# First things first

Get the data from: https://s3.amazonaws.com/soundflux-urbansounds/curated_4_class_bundle.zip

# Create image generators

In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
set_session(sess)

Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import json
from PIL import Image

In [3]:
def generate_generator_multiple(generator,directories, batch_size, img_height,img_width):
    generators =[]
    for directory in directories:
        gen = generator.flow_from_directory(directory,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=True, 
                                          seed=7)
    
        generators.append(gen)

    for gen in generators:
        for data, labels in gen:
            yield data, labels

In [4]:
def save_plots(history,target_file_acc,target_file_loss):
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(target_file_acc)
    plt.close()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(target_file_loss)
    plt.close()

In [ ]:
def show_plots(history):
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

# Doing K-Folds

In [ ]:
batch_size = 32
img_height=80
img_width = 256
approx_fold_size = 1000

In [ ]:
train_folder = "/home/nvidia/bundle/split/train/"
test_folder = "/home/nvidia/bundle/split/test/"

# MODEL

In [ ]:
datagen = ImageDataGenerator(rescale=1./255,
                            #rotation_range=10,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.2,
                            zoom_range=0.3,
                            #horizontal_flip=True,
                            #vertical_flip=True,
                            fill_mode='nearest')

In [ ]:
input_shape = (img_height, img_width,3)
nclass = 5

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers

Next ideas:

- Add filters on time with strides! 2,1 convolutions with padding and a gap
- Keep at least a good 64 channels open?
- Sum pooling instead of averaging?

# try 

keras.applications.densenet.DenseNet121 next

In [ ]:
base_model = keras.applications.vgg16.VGG16(weights='imagenet', 
                                include_top=False, 
                                input_shape=(img_height, img_width,3))
base_model.trainable = False
model = keras.models.Sequential()
model.add(base_model)
model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Dense(512,activation='relu'))
model.add(keras.layers.Dense(64,activation='relu'))
model.add(keras.layers.Dense(32,activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(nclass, activation='softmax'))

In [ ]:
opt = optimizers.RMSprop(lr=0.001)
#opt = optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', 
              optimizer=opt,
              metrics=['accuracy'])
#needed to reset weigh"""ts!
model.save_weights('raw_model.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 2, 8, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 165       
Total para

**LSTM Sizing**
This is helpful when understanding sizing
https://stackoverflow.com/questions/50242925/data-preprocessing-input-shape-for-timedistributed-cnn-lrcn-convlstm2d-for?rq=1

**Explore**
- Should also be returning state?
- Should also flip sizing? (channel representation should be 3 instead of the height?)
- Run another LSTM from top to bottom to traverse from the spectrums/frequency bands from lower to higher!?!

# Data Generator

In [ ]:
train_generator = generate_generator_multiple(generator=datagen,
                                           directories = [train_folder],
                                           batch_size=batch_size,
                                           img_height=img_height,
                                           img_width=img_width)
test_generator = generate_generator_multiple(generator=datagen,
                                           directories = [test_folder],
                                           batch_size=batch_size,
                                           img_height=img_height,
                                           img_width=img_width)

https://towardsdatascience.com/image-detection-from-scratch-in-keras-f314872006c9

In [ ]:
#RESET WEIGHTS!!
#model.load_weights('raw_model.h5')
#
history = model.fit_generator(train_generator,
                          steps_per_epoch=approx_fold_size/batch_size,
                          validation_data = test_generator,
                          validation_steps = approx_fold_size/batch_size,
                          epochs=30,
                          shuffle=True, 
                          verbose=True)

Epoch 1/30
Found 432 images belonging to 5 classes.
 9/31 [=======>......................] - ETA: 54s - loss: 1.6080 - acc: 0.2465 